In [16]:
import boto3
import sagemaker
import time


region = boto3.Session().region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker.session.Session().default_bucket()
s3_client = boto3.client("sagemaker")

bucket_name = "deepfake-detection"
dataset_prefix = "dev_datasets"
output_prefix = f"{dataset_prefix}/preprocessed_data"
model_package_group_name = "deepfake-detection-models"  
pipeline_name = "Deepfake-detection-pipeline"  

## Get the pipeline instance

In [ ]:
from pipelines.pipeline import get_pipeline


pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    bucket_name=bucket_name,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    output_prefix=output_prefix
)

## Submit the pipeline to SageMaker and start execution

In [ ]:
pipeline.upsert(role_arn=role)

In [5]:
execution = pipeline.start()

### Pipeline Operations: examining and waiting for pipeline execution

In [8]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:674518009863:pipeline/deepfake-detection-pipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:674518009863:pipeline/deepfake-detection-pipeline/execution/nde0z3j7layb',
 'PipelineExecutionDisplayName': 'execution-1672869399477',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'deepfake-detection-pipeline',
  'TrialName': 'nde0z3j7layb'},
 'CreationTime': datetime.datetime(2023, 1, 4, 21, 56, 39, 336000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 1, 4, 21, 56, 39, 336000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:674518009863:user-profile/d-lhlxnd4plzw6/nante',
  'UserProfileName': 'nante',
  'DomainId': 'd-lhlxnd4plzw6'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:674518009863:user-profile/d-lhlxnd4plzw6/nante',
  'UserProfileName': 'nante',
  'DomainId': 'd-lhlxnd4plzw6'},
 'ResponseMetadata': {'RequestId'

We can list the execution steps to check out the status and artifacts:

In [10]:
execution.list_steps()

[{'StepName': 'deepfake-detection-RegisterModel',
  'StartTime': datetime.datetime(2023, 1, 4, 22, 10, 44, 394000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 1, 4, 22, 10, 45, 514000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:674518009863:model-package/deepfake-detection-models/3'}}},
 {'StepName': 'check-accuracy-deepfake-evaluation',
  'StartTime': datetime.datetime(2023, 1, 4, 22, 10, 43, 419000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 1, 4, 22, 10, 43, 808000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Condition': {'Outcome': 'True'}}},
 {'StepName': 'evaluate-deepfake-detection-model',
  'StartTime': datetime.datetime(2023, 1, 4, 22, 5, 49, 857000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 1, 4, 22, 10, 42, 979000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Proc

## Clean up (optional)
Delete the model registry and the pipeline to keep the studio environment tidy.

In [12]:
def delete_model_package_group(sm_client, package_group_name):
    try:
        model_versions = sm_client.list_model_packages(ModelPackageGroupName=package_group_name)

    except Exception as e:
        print("{} \n".format(e))
        return

    for model_version in model_versions["ModelPackageSummaryList"]:
        try:
            sm_client.delete_model_package(ModelPackageName=model_version["ModelPackageArn"])
        except Exception as e:
            print("{} \n".format(e))
        time.sleep(0.5)  # Ensure requests aren't throttled

    try:
        sm_client.delete_model_package_group(ModelPackageGroupName=package_group_name)
        print("{} model package group deleted".format(package_group_name))
    except Exception as e:
        print("{} \n".format(e))
    return


def delete_sagemaker_pipeline(sm_client, pipeline_name):
    try:
        sm_client.delete_pipeline(
            PipelineName=pipeline_name,
        )
        print("{} pipeline deleted".format(pipeline_name))
    except Exception as e:
        print("{} \n".format(e))
        return

In [17]:
delete_model_package_group(s3_client, model_package_group_name)
delete_sagemaker_pipeline(s3_client, pipeline_name)

deepfake-detection-models model package group deleted
Deepfake-detection-pipeline pipeline deleted
